In [ ]:
import speech_recognition as sr
import sys
from os import path
import os
from moviepy.editor import *
from google.protobuf.json_format import MessageToJson
import json
from google.cloud import speech_v1
import pandas as pd
from gcloud import storage

def generateURIList():
    uri_list = []    
    client = storage.Client()
    BUCKET_NAME = 'files_audio'
    bucket = client.get_bucket(BUCKET_NAME)
    blobs = bucket.list_blobs()

    for key in blobs:
        if "wav/" in key.name:
            uri_list.append(key.name)
    return(uri_list)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="creds.json"


def sample_long_running_recognize(uri):
    client = speech_v1.SpeechClient()
    storage_uri = "gs://files_audio/" + uri
    #wav/test_audio.wav'

    # When enabled, the first result returned by the API will include a list
    # of words and the start and end time offsets (timestamps) for those words.
    enable_word_time_offsets = True

    # The language of the supplied audio
    language_code = "en-US"
    config = {
        "enable_word_time_offsets": enable_word_time_offsets,
        "language_code": language_code,
        "audio_channel_count" : 2
    }
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()
    return(response)

In [ ]:
urilist = generateURIList()
response = sample_long_running_recognize(urilist[-1])

In [ ]:
serialized = MessageToJson(response)
jsonOutput = json.loads(serialized)

In [ ]:
#with open('sample.json') as json_file:
#    data = json.load(json_file)

In [ ]:
df = pd.DataFrame(columns=['filename','startTime','endTime','word'])

for elem in jsonOutput['results']:
    for word in elem['alternatives'][0]['words']:
        startTime = word['startTime']
        endTime = word['endTime']
        slowo = word['word']
        df = df.append({'filename':"asd", 'startTime':startTime, 'endTime':endTime, 'word':slowo},ignore_index=True)

In [ ]:
df